In [ ]:
#Comparing Scores: Parametric

In [106]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import xarray as xr
import matplotlib.pyplot as plt
from src.data_generator import *
from src.train import *
from src.utils import *
from src.networks import *

In [108]:
os.environ["CUDA_VISIBLE_DEVICES"]=str(0)
limit_mem()

In [53]:
exp_id_path='/home/garg/WeatherBench/nn_configs/B/81-resnet_d3_dr_0.1.yml' #Change it
model_save_dir='/home/garg/data/WeatherBench/predictions/saved_models'
datadir='/home/garg/data/WeatherBench/5.625deg'
pred_save_dir='/home/garg/data/WeatherBench/predictions'

In [49]:
preds_parametric=xr.open_dataset(f'{pred_save_dir}/100-resnet_d3_param.nc')
preds_parametric=preds_parametric.rename({'z-mean':'z_mean','z-std':'z_std', 't-mean':'t_mean', 't-std':'t_std'})
preds_parametric

In [140]:
#Observation
z500_valid = load_test_data(f'{datadir}/geopotential_500', 'z')
t850_valid = load_test_data(f'{datadir}/temperature_850', 't')

observation=xr.Dataset()
observation['z_500']=z500_valid
observation['t_850']=t850_valid

observation=observation.astype('float64')
observation=observation.drop('level')

In [ ]:
import xskillscore as xs

In [83]:
preds_parametric_mean=preds_parametric[['z_mean','t_mean']].rename({'z_mean':'z_500','t_mean':'t_850'})
preds_parametric_std=preds_parametric[['z_std','t_std']].rename({'z_std':'z_500','t_std':'t_850'})

In [112]:
def compute_weighted_crps_gaussian(obs, mean, std,mean_dims=xr.ALL_DIMS):
    weights_lat = np.cos(np.deg2rad(obs.lat)).values
    weights_lat /= weights_lat.mean()
    crps = xs.crps_gaussian(obs,mean,std)
    crps = (crps * weights_lat[None, : , None]).mean(mean_dims)
    return crps

In [113]:
crps=compute_weighted_crps_gaussian(observation.sel(time=preds_parametric.time), 
                                    preds_parametric_mean, preds_parametric_std)

In [129]:
crps.load()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z_500    float64 1.256e+04
    t_850    float64 63.7

In [133]:
#RMSE
rmse=compute_weighted_rmse(preds_parametric_mean, observation)

In [135]:
rmse.load()

<xarray.Dataset>
Dimensions:     ()
Data variables:
    z_500_rmse  float64 414.4
    t_850_rmse  float64 2.236

In [ ]:
#COMPARING WITH Ensemble SCORES

In [136]:
exp_id='81-resnet_d3_dr_0.1'
preds_ensemble=xr.open_dataset(f'{pred_save_dir}/{exp_id}.nc')

In [142]:
crps=compute_weighted_crps_gaussian(observation.sel(time=preds_ensemble.time), 
                                    preds_ensemble.mean('member'), preds_ensemble.std('member'))

In [143]:
crps.load()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z_500    float64 160.1
    t_850    float64 1.053

In [144]:
rmse=compute_weighted_rmse(preds_ensemble.mean('member'),observation)

In [145]:
rmse.load()

<xarray.Dataset>
Dimensions:     ()
Data variables:
    z_500_rmse  float64 317.9
    t_850_rmse  float64 1.837

In [ ]:
#So pretty bad crps score. 1 magnitude higher.